# 1. Data loading and preprocessing

In [1]:
import pandas as pd

In [4]:
directory = 'clustersAtractivoJuntos_2023.csv'
df = pd.read_csv(directory)
df["grupo"] = df["grupo"].fillna(1)
print(df.head())

              Nombre  escuelas/100_alumnos_priv  escuelas/100_alumnos_pub  \
0       Acebeda (La)                  49.098317                 39.938596   
1  Alameda del Valle                  49.098317                 39.938596   
2             Ambite                  49.098317                 79.979532   
3           Anchuelo                  49.098317                 42.064663   
4        Atazar (El)                  49.098317                 39.938596   

   unidadEsc/100_alumnos_priv  unidadEsc/100_alumnos_pub  \
0                   48.729767                  29.490332   
1                   48.729767                  29.490332   
2                   48.729767                  68.662370   
3                   48.729767                  56.529438   
4                   48.729767                  29.490332   

   profesores/100_alumnos_pub  profesores/100_alumnos_priv  \
0                   30.553000                    48.467897   
1                   30.553000                    48.

In [5]:
len(df)

178

In [6]:
df.iloc(0)

In [7]:
blocks = {
    "educacion": ['escuelas/100_alumnos_priv', 'escuelas/100_alumnos_pub',
                   'unidadEsc/100_alumnos_priv', 'unidadEsc/100_alumnos_pub',
                   'profesores/100_alumnos_pub', 'profesores/100_alumnos_priv',
                   'bibliotecas/100_alumnos', 'idiomas/10_alumnos',
                   'actividades/10_alumnos'],
    "airquality": ['monoxidoCarbono', 'monoxidoNitrogeno',
                    'dioxidoNitrogeno', 'pm10', 'oxidosNitrogeno', 'ozono',
                    'temperatura', 'radiacionSolar', 'precipitacion'],
    "tipologia": ['Total_pop', 'frac_extranjeros', 'frac_young', 'frac_old',
                   'frac_HighEduc', 'edadMedia', 'densidad'],
    "housing": ['familiares',
                 'antiguedadFamiliares', 'superficieFamiliares',
                 'unidadesUrbanasResidenciales', '1-3p', '4+p', 'size', 'rooms',
                 'price', 'priceByArea', 'bathrooms', 'hasLift', 'parkingSpace',
                 'newDevelopment', 'propertyType', 'pisoBajo', 'numHouses'],
    "salud": ['centros_100', 'farmacias_100', 'centrosSociales_10',
               'clinicaDental_10','consultaPrimaria_10', 'otroConsulta_10', 'consultaPrimaria_10',
               'orgNoSanitaria_10'],
    "transporte": ['distanciaCentro', 'bus_100', 'estaciontren_100',
                    'servicioCoches_100', 'estacionBus_100'],
    "economia": ['paro_100', 'rentaBruta',
                  'pensionMedia', 'trabajadores_100', 'contratos_100',
                  'pensionistas_100', 'ocupadosColectivos_100',
                  'funcionarios_100', 'organizaciones_100']
}

# Columnas a evaluar (Higher significa que cuanto más mejor, se dejan igual. Lower que cuanto menos mejor, se tienen que invertir)
column_types = {
    "educacion": {"colsHigher": blocks["educacion"], "colsLower": []},
    "tipologia": {"colsHigher": ['frac_extranjeros', 'frac_young', 'frac_HighEduc', 'Total_pop'],
                  "colsLower": ['edadMedia', 'densidad', 'frac_old']},
    "housing": {"colsHigher": ['familiares', 'superficieFamiliares', 'unidadesUrbanasResidenciales', '1-3p', '4+p', 'size', 'rooms', 'bathrooms',
                               'hasLift', 'parkingSpace', 'newDevelopment', 'propertyType', 'pisoBajo', 'numHouses'],
                "colsLower": ['price', 'priceByArea', 'antiguedadFamiliares']},
    "salud": {"colsHigher": blocks["salud"], "colsLower": []},
    "transporte": {"colsHigher": ['bus_100', 'estaciontren_100', 'servicioCoches_100', 'estacionBus_100'],
                   "colsLower": ['distanciaCentro']},
    "economia": {"colsHigher": [ 'pensionMedia', 'trabajadores_100', 'contratos_100',
                                'ocupadosColectivos_100', 'funcionario_100', 'organizaciones_100'],
                 "colsLower": ['paro_100', 'pensionistas_100']},
    "airquality": {"colsHigher": [], "colsLower": blocks["airquality"]}
}

In [8]:
# Iterar sobre los tipos de columnas
for block, columns in column_types.items():
    # Obtener las columnas que necesitan inversa
    cols_lower = columns["colsLower"]
    print("cols_lower of ", block, ":", cols_lower)
    # Aplicar la transformación
    for col in cols_lower:
        if col in df.columns:  # Verificar que la columna exista en el DataFrame
            df[col] = 100 - df[col]

cols_lower of  educacion : []
cols_lower of  tipologia : ['edadMedia', 'densidad', 'frac_old']
cols_lower of  housing : ['price', 'priceByArea', 'antiguedadFamiliares']
cols_lower of  salud : []
cols_lower of  transporte : ['distanciaCentro']
cols_lower of  economia : ['paro_100', 'pensionistas_100']
cols_lower of  airquality : ['monoxidoCarbono', 'monoxidoNitrogeno', 'dioxidoNitrogeno', 'pm10', 'oxidosNitrogeno', 'ozono', 'temperatura', 'radiacionSolar', 'precipitacion']


# 2. Data normalization

Los intervalos son con los datos del mismo municipio. La normalización de valor / (max-min) es con los valores individuales de todos los municipios.

De tal manera que si tenemos:

| Municipio   | Indicador 1 (Bloque A) | Indicador 2 (Bloque A) | Indicador 3 (Bloque B) | Indicador 4 (Bloque B) |
|-------------|-------------------------|-------------------------|-------------------------|-------------------------|
| Municipio 1 | 78                      | 45                      | 62                      | 89                      |
| Municipio 2 | 80                      | 56                      | 91                      | 73                      |


El municipio 1, bloque A estaría entre el intervalo [45, (45+78)/2] pero para normalizarlo usaríamos como min el 45 y máximo el 80 del municipio 2.

## 2.1. Intervalos por bloque

In [9]:
def calculate_intervals(df, blocks):
    results = []

    for municipio, group in df.groupby('Nombre'):
        # Obtener el grupo asociado al municipio
        grupo_value = group['grupo'].iloc[0] if 'grupo' in group.columns else None

        municipio_result = {'Nombre': municipio, 'grupo': grupo_value}

        for block_name, columns in blocks.items():
            # Filtrar las columnas del bloque que están en el DataFrame, excluyendo "grupo"
            block_columns = [col for col in columns if col in df.columns and col != 'grupo']

            if block_columns:  # Asegurarse de que hay columnas válidas
                block_data = group[block_columns]

                # Calcular el mínimo y la media
                min_value = block_data.min().min()
                mean_value = block_data.mean().mean()

                municipio_result[block_name] = [min_value, mean_value]
            else:
                municipio_result[block_name] = None  # No hay datos para este bloque

        results.append(municipio_result)

    return pd.DataFrame(results)

intervalos_df = calculate_intervals(df, blocks)
intervalos_df.head()

,Nombre,grupo,educacion,airquality,tipologia,housing,salud,transporte,economia
0,Acebeda (La),1.0,"[29.49033168245857, 43.79609323039056]","[40.50182121447103, 54.505621376106625]","[0.4578734667923856, 49.82551711510932]","[0.0, 48.56831806504307]","[37.86330364170312, 57.29596208778984]","[48.737630310809, 71.49290172324913]","[46.860285028032735, 76.13598651008814]"
1,Ajalvir,2.0,"[31.86116753967628, 44.84494552591675]","[27.132332423861072, 62.63764295250847]","[28.918254207601166, 46.34356847351074]","[18.5271664724569, 43.08911685839731]","[36.20907228919217, 45.947637179195226]","[30.57627745062433, 39.07336718797229]","[30.576277450624332, 63.65561580254467]"
2,Alameda del Valle,1.0,"[29.49033168245857, 43.79609323039056]","[40.50182121447103, 54.505621376106625]","[27.840532051376577, 53.23725574358504]","[25.57234295505057, 57.21220105375252]","[44.68278662862529, 58.204971074990105]","[48.737630310809, 63.75954747666595]","[41.448093453824114, 52.86223168221622]"
3,Alamo (El),2.0,"[41.892246409108914, 47.828773505431556]","[8.13738026539518, 62.57171248862591]","[45.229774454143566, 54.52373773504791]","[33.537363883511205, 46.540529140085795]","[34.28315683161047, 43.28820487063598]","[46.26168498135839, 48.65376799557049]","[30.117085568662727, 43.214425920342535]"
4,Alcalá de Henares,3.0,"[32.17767100517428, 47.969169391905616]","[25.32591640605915, 57.060275485193706]","[27.508918522070967, 60.09053374864954]","[20.455648084992102, 53.520042858136314]","[48.27751166670492, 60.01140153367968]","[51.80805877012301, 62.91185515993038]","[38.336731564368606, 49.669340651239565]"


In [10]:
intervalos_df[intervalos_df["Nombre"] == "Quijorna"]

,Nombre,grupo,educacion,airquality,tipologia,housing,salud,transporte,economia
114,Quijorna,2.0,"[33.15469153366796, 76.67283043920412]","[0.557636140306883, 25.99358594719894]","[35.940133190427275, 48.09915602462735]","[28.53092774394087, 48.993216831246606]","[41.72866877848614, 51.70037109212258]","[38.95731763856869, 45.22216911016329]","[38.95731763856869, 44.68617351797365]"


## 2.2. Valor único por bloque

In [11]:
valores_unicos_df = pd.DataFrame(index=df.index)  # Asegurarse de que el índice sea el mismo que df

# Crear un diccionario para almacenar los valores mínimo y máximo por bloque y por grupo
min_max_by_block_and_group = {}

# Iterar sobre los grupos
for group in df['grupo'].unique():
    group_data = df[df['grupo'] == group]  # Filtrar los datos del grupo actual

    # Crear un diccionario para almacenar los resultados por bloque
    min_max_by_block = {}

    print(f"Procesando grupo {group}...")  # Print para verificar qué grupo estamos procesando

    # Calcular la media por bloque para cada municipio
    for block, indicators in blocks.items():
        # Seleccionar solo las columnas relevantes para el bloque que están en el DataFrame
        relevant_columns = [col for col in indicators if col in group_data.columns]

        if not relevant_columns:  # Si no hay columnas relevantes, saltar este bloque
            print(f"  No se encontraron columnas relevantes para el bloque {block} en el grupo {group}.")
            continue

        # Calcular la media para cada fila (municipio)
        mean_values = group_data[relevant_columns].mean(axis=1)

        # Normalizar las medias por el rango máximo-mínimo del bloque
        block_min = group_data[relevant_columns].min().min()  # Mínimo global del bloque
        block_max = group_data[relevant_columns].max().max()  # Máximo global del bloque

        if block_max == block_min:
            print(f"  Advertencia: El rango del bloque {block} para el grupo {group} es cero (max = min).")

        normalized_mean_values = (mean_values) / (block_max - block_min)

        # Asegurarse de que los índices coinciden al asignar los valores normalizados
        valores_unicos_df.loc[group_data.index, block] = normalized_mean_values

        # Almacenar los valores mínimo y máximo para el bloque
        min_max_by_block[block] = {
            "min": block_min,
            "max": block_max
        }

    # Almacenar los resultados para el grupo actual
    min_max_by_block_and_group[group] = min_max_by_block

# Mantener la columna 'grupo' en el DataFrame
valores_unicos_df['grupo'] = df['grupo']

# Añadir la columna 'Nombre' al nuevo DataFrame
valores_unicos_df['Nombre'] = df['Nombre']

# Reordenar las columnas para que 'Nombre' y 'grupo' estén al principio
valores_unicos_df = valores_unicos_df[['Nombre', 'grupo'] + list(valores_unicos_df.columns[:-2])]

# Mostrar los valores de mínimo y máximo por bloque y grupo
print("\nResultados finales:")
for group, blocks in min_max_by_block_and_group.items():
    print(f"Grupo {group}:")
    for block_name, values in blocks.items():
        print(f"  {block_name}:")
        print(f"    Mínimo = {values['min']}")
        print(f"    Máximo = {values['max']}")

# Mostrar el nuevo DataFrame con las medias normalizadas
print("\nDataFrame final con medias normalizadas:")
print(valores_unicos_df)

Procesando grupo 1.0...
Procesando grupo 3.0...
Procesando grupo 2.0...

Resultados finales:
Grupo 1.0:
  educacion:
    Mínimo = 29.49033168245857
    Máximo = 100.0
  airquality:
    Mínimo = 0.0
    Máximo = 100.0
  tipologia:
    Mínimo = 0.0
    Máximo = 100.0
  housing:
    Mínimo = 0.0
    Máximo = 100.0
  salud:
    Mínimo = 37.58539639755824
    Máximo = 100.0
  transporte:
    Mínimo = 5.47850803877184
    Máximo = 100.0
  economia:
    Mínimo = 0.0
    Máximo = 100.0
Grupo 3.0:
  educacion:
    Mínimo = 0.0
    Máximo = 100.0
  airquality:
    Mínimo = 0.0
    Máximo = 100.0
  tipologia:
    Mínimo = 0.0
    Máximo = 100.0
  housing:
    Mínimo = 0.0
    Máximo = 100.0
  salud:
    Mínimo = 0.0
    Máximo = 100.0
  transporte:
    Mínimo = 4.018329395189757
    Máximo = 100.0
  economia:
    Mínimo = 0.0
    Máximo = 100.0
Grupo 2.0:
  educacion:
    Mínimo = 25.161450961900663
    Máximo = 100.0
  airquality:
    Mínimo = 0.0
    Máximo = 100.0
  tipologia:
    Mínimo = 0.0

## 2.3. Intervalos totales

In [12]:
def add_total_interval(valores_unicos_df, intervalos_df):
    # Excluir las columnas "Municipio" y "grupo" antes de calcular el intervalo
    filtered_df = valores_unicos_df.drop(columns=['Nombre', 'grupo'], errors='ignore')

    # Calcular [min, media] por fila
    total_intervals = filtered_df.apply(lambda row: [row.min(), row.mean()], axis=1)

    # Añadir la nueva columna 'total' al DataFrame intervalos_df
    intervalos_df['total'] = total_intervals.values

    return intervalos_df

intervalos_df = add_total_interval(valores_unicos_df, intervalos_df)
intervalos_df.head()

,Nombre,grupo,educacion,airquality,tipologia,housing,salud,transporte,economia,total
0,Acebeda (La),1.0,"[29.49033168245857, 43.79609323039056]","[40.50182121447103, 54.505621376106625]","[0.4578734667923856, 49.82551711510932]","[0.0, 48.56831806504307]","[37.86330364170312, 57.29596208778984]","[48.737630310809, 71.49290172324913]","[46.860285028032735, 76.13598651008814]","[0.4856831806504308, 0.6551209620292816]"
1,Ajalvir,2.0,"[31.86116753967628, 44.84494552591675]","[27.132332423861072, 62.63764295250847]","[28.918254207601166, 46.34356847351074]","[18.5271664724569, 43.08911685839731]","[36.20907228919217, 45.947637179195226]","[30.57627745062433, 39.07336718797229]","[30.576277450624332, 63.65561580254467]","[0.5286223168221622, 0.6294876594449121]"
2,Alameda del Valle,1.0,"[29.49033168245857, 43.79609323039056]","[40.50182121447103, 54.505621376106625]","[27.840532051376577, 53.23725574358504]","[25.57234295505057, 57.21220105375252]","[44.68278662862529, 58.204971074990105]","[48.737630310809, 63.75954747666595]","[41.448093453824114, 52.86223168221622]","[0.44882420516173716, 0.5613977300452124]"
3,Alamo (El),2.0,"[41.892246409108914, 47.828773505431556]","[8.13738026539518, 62.57171248862591]","[45.229774454143566, 54.52373773504791]","[33.537363883511205, 46.540529140085795]","[34.28315683161047, 43.28820487063598]","[46.26168498135839, 48.65376799557049]","[30.117085568662727, 43.214425920342535]","[0.3621680994383565, 0.5543473926538552]"
4,Alcalá de Henares,3.0,"[32.17767100517428, 47.969169391905616]","[25.32591640605915, 57.060275485193706]","[27.508918522070967, 60.09053374864954]","[20.455648084992102, 53.520042858136314]","[48.27751166670492, 60.01140153367968]","[51.80805877012301, 62.91185515993038]","[38.336731564368606, 49.669340651239565]","[0.4427309185439901, 0.5893095732994064]"


## 2.4. Valores individuales totales

In [13]:
valores_unicos_df

,Nombre,grupo,educacion,airquality,tipologia,housing,salud,transporte,economia
0,Acebeda (La),1.0,0.621136,0.545056,0.498255,0.485683,0.917990,0.756367,0.761360
1,Alameda del Valle,1.0,0.621136,0.545056,0.532373,0.572122,0.932554,0.674551,0.528622
2,Ambite,1.0,0.831079,0.450766,0.483403,0.448824,0.762807,0.464846,0.488058
3,Anchuelo,1.0,0.696472,0.638144,0.490801,0.503888,0.710397,0.362168,0.478562
4,Atazar (El),1.0,0.621136,0.545056,0.449757,0.442731,0.810454,0.656462,0.599571
...,...,...,...,...,...,...,...,...,...
173,Villalbilla,2.0,0.580331,0.626376,0.462266,0.519048,0.508697,0.386864,0.443544
174,Villanueva de la Cañada,2.0,0.580923,0.555602,0.636872,0.624040,0.608939,0.396010,0.554094
175,Villanueva del Pardillo,2.0,0.598664,0.555602,0.537714,0.531436,0.604745,0.365587,0.480469
176,Villarejo de Salvanés,2.0,0.596529,0.386241,0.455873,0.469883,0.534913,0.641240,0.527485


In [14]:
# 1. Calcular la media por fila (excluyendo 'Municipio' y 'grupo')
valores_unicos_df['mean_per_row'] = valores_unicos_df.drop(['Nombre', 'grupo'], axis=1).mean(axis=1)  # media por fila

# 2. Calcular el mínimo y máximo global por grupo (excluyendo 'mean_per_row')
# Seleccionar las columnas restantes sin contar 'Municipio', 'grupo', y 'mean_per_row'
columns_of_interest = valores_unicos_df.drop(columns=['Nombre', 'grupo', 'mean_per_row']).columns

# Calcular el mínimo y máximo de cada columna de interés por grupo
group_min = valores_unicos_df.groupby('grupo')[columns_of_interest].transform('min')
group_max = valores_unicos_df.groupby('grupo')[columns_of_interest].transform('max')

# 3. Normalizar la media por fila usando los valores máximos y mínimos globales por grupo
# Usamos group_min y group_max para cada grupo
# La normalización es: (mean_per_row - min del grupo) / (max del grupo - min del grupo)
valores_unicos_df['total'] = (valores_unicos_df['mean_per_row']) / (group_max.max(axis=1) - group_min.min(axis=1))

# 4. Opcional: Drop la columna auxiliar 'mean_per_row' si ya no la necesitas
# valores_unicos_df = valores_unicos_df.drop(columns=['mean_per_row'], errors='ignore')

# Mostrar los primeros resultados
valores_unicos_df.head()


,Nombre,grupo,educacion,airquality,tipologia,housing,salud,transporte,economia,mean_per_row,total
0,Acebeda (La),1.0,0.621136,0.545056,0.498255,0.485683,0.917990,0.756367,0.761360,0.655121,0.930889
1,Alameda del Valle,1.0,0.621136,0.545056,0.532373,0.572122,0.932554,0.674551,0.528622,0.629488,0.894465
2,Ambite,1.0,0.831079,0.450766,0.483403,0.448824,0.762807,0.464846,0.488058,0.561398,0.797713
3,Anchuelo,1.0,0.696472,0.638144,0.490801,0.503888,0.710397,0.362168,0.478562,0.554347,0.787695
4,Atazar (El),1.0,0.621136,0.545056,0.449757,0.442731,0.810454,0.656462,0.599571,0.589310,0.837374


# Exports

In [15]:
intervalos_df.head()

,Nombre,grupo,educacion,airquality,tipologia,housing,salud,transporte,economia,total
0,Acebeda (La),1.0,"[29.49033168245857, 43.79609323039056]","[40.50182121447103, 54.505621376106625]","[0.4578734667923856, 49.82551711510932]","[0.0, 48.56831806504307]","[37.86330364170312, 57.29596208778984]","[48.737630310809, 71.49290172324913]","[46.860285028032735, 76.13598651008814]","[0.4856831806504308, 0.6551209620292816]"
1,Ajalvir,2.0,"[31.86116753967628, 44.84494552591675]","[27.132332423861072, 62.63764295250847]","[28.918254207601166, 46.34356847351074]","[18.5271664724569, 43.08911685839731]","[36.20907228919217, 45.947637179195226]","[30.57627745062433, 39.07336718797229]","[30.576277450624332, 63.65561580254467]","[0.5286223168221622, 0.6294876594449121]"
2,Alameda del Valle,1.0,"[29.49033168245857, 43.79609323039056]","[40.50182121447103, 54.505621376106625]","[27.840532051376577, 53.23725574358504]","[25.57234295505057, 57.21220105375252]","[44.68278662862529, 58.204971074990105]","[48.737630310809, 63.75954747666595]","[41.448093453824114, 52.86223168221622]","[0.44882420516173716, 0.5613977300452124]"
3,Alamo (El),2.0,"[41.892246409108914, 47.828773505431556]","[8.13738026539518, 62.57171248862591]","[45.229774454143566, 54.52373773504791]","[33.537363883511205, 46.540529140085795]","[34.28315683161047, 43.28820487063598]","[46.26168498135839, 48.65376799557049]","[30.117085568662727, 43.214425920342535]","[0.3621680994383565, 0.5543473926538552]"
4,Alcalá de Henares,3.0,"[32.17767100517428, 47.969169391905616]","[25.32591640605915, 57.060275485193706]","[27.508918522070967, 60.09053374864954]","[20.455648084992102, 53.520042858136314]","[48.27751166670492, 60.01140153367968]","[51.80805877012301, 62.91185515993038]","[38.336731564368606, 49.669340651239565]","[0.4427309185439901, 0.5893095732994064]"


In [16]:
valores_unicos_df.head()

,Nombre,grupo,educacion,airquality,tipologia,housing,salud,transporte,economia,mean_per_row,total
0,Acebeda (La),1.0,0.621136,0.545056,0.498255,0.485683,0.917990,0.756367,0.761360,0.655121,0.930889
1,Alameda del Valle,1.0,0.621136,0.545056,0.532373,0.572122,0.932554,0.674551,0.528622,0.629488,0.894465
2,Ambite,1.0,0.831079,0.450766,0.483403,0.448824,0.762807,0.464846,0.488058,0.561398,0.797713
3,Anchuelo,1.0,0.696472,0.638144,0.490801,0.503888,0.710397,0.362168,0.478562,0.554347,0.787695
4,Atazar (El),1.0,0.621136,0.545056,0.449757,0.442731,0.810454,0.656462,0.599571,0.589310,0.837374


In [18]:
from os import mkdir


intervalos_df.to_csv("atractividad_2023_final/intervalos_df2.csv", index = False)
valores_unicos_df.to_csv("atractividad_2023_final/valores_unicos_df2.csv", index = False)